In [1]:
#pip install --quiet --upgrade rich

In [2]:
#pip install --quiet --upgrade streamlit

In [3]:
#pip install --quiet --upgrade datasets

In [4]:
#pip install --quiet --upgrade sentence_transformers

In [5]:
#pip install --quiet --upgrade torchvision

In [6]:
#pip install --quiet --upgrade torch

In [7]:
#pip install --quiet --upgrade pinecone-client

In [8]:
#pip install --quiet --upgrade pinecone-datasets

In [9]:
#pip install --quiet --upgrade pinecone-text

In [10]:
#pip install --quiet --upgrade tqdm

In [11]:
#pip install --quiet --upgrade tiktoken

In [12]:
#pip install --quiet --upgrade deepface

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
#pip install tf-keras==2.17.0

In [15]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

In [16]:
from pinecone import Pinecone, ServerlessSpec
import os
import time
import torch

In [17]:
from UDCUtils import UDCUtils

utils = UDCUtils()

In [18]:
pinecone_api_key = utils.get_pinecone_api_key()
#print(type(pinecone_api_key))

if pinecone_api_key==None:
    pinecone_api_key = "a85f0b7c-6ce3-458d-aa18-168f39e4420b" ##default key

In [19]:
print(pinecone_api_key)

a85f0b7c-6ce3-458d-aa18-168f39e4420b


In [20]:
from tqdm.auto import tqdm

In [46]:
import datasets

#datasets.get_dataset_infos(path="quora")

datasets.list_datasets()

['amirveyseh/acronym_identification',
 'ade-benchmark-corpus/ade_corpus_v2',
 'UCLNLP/adversarial_qa',
 'Yale-LILY/aeslc',
 'nwu-ctext/afrikaans_ner_corpus',
 'fancyzhx/ag_news',
 'allenai/ai2_arc',
 'google/air_dialogue',
 'komari6/ajgt_twitter_ar',
 'legacy-datasets/allegro_reviews',
 'tblard/allocine',
 'mutiyama/alt',
 'fancyzhx/amazon_polarity',
 'defunct-datasets/amazon_reviews_multi',
 'defunct-datasets/amazon_us_reviews',
 'sewon/ambig_qa',
 'nala-cub/americas_nli',
 'legacy-datasets/ami',
 'gavinxing/amttl',
 'facebook/anli',
 'sealuzh/app_reviews',
 'deepmind/aqua_rat',
 'google-research-datasets/aquamuse',
 'bigIR/ar_cov19',
 'hadyelsahar/ar_res_reviews',
 'iabufarha/ar_sarcasm',
 'abuelkhair-corpus/arabic_billion_words',
 'QCRI/arabic_pos_dialect',
 'halabi2016/arabic_speech_corpus',
 'hsseinmz/arcd',
 'ramybaly/arsentd_lev',
 'allenai/art',
 'arxiv-community/arxiv_dataset',
 'tuanphong/ascent_kb',
 'achrafothman/aslg_pc12',
 'AmazonScience/asnq',
 'facebook/asset',
 'nilc-

In [21]:
dataset = load_dataset('quora', split='train[:10000]', trust_remote_code=True)

In [22]:
dataset[:5]

{'questions': [{'id': [1, 2],
   'text': ['What is the step by step guide to invest in share market in india?',
    'What is the step by step guide to invest in share market?']},
  {'id': [3, 4],
   'text': ['What is the story of Kohinoor (Koh-i-Noor) Diamond?',
    'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?']},
  {'id': [5, 6],
   'text': ['How can I increase the speed of my internet connection while using a VPN?',
    'How can Internet speed be increased by hacking through DNS?']},
  {'id': [7, 8],
   'text': ['Why am I mentally very lonely? How can I solve it?',
    'Find the remainder when [math]23^{24}[/math] is divided by 24,23?']},
  {'id': [9, 10],
   'text': ['Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?',
    'Which fish would survive in salt water?']}],
 'is_duplicate': [False, False, False, False, False]}

In [23]:
#read all questions in a list
questions = []

for record in dataset["questions"]:
    questions.extend(record['text'])

#convert list into set of unique questions
question_lst = list(set(questions))

In [24]:
print('-'*50)
print("\n".join(question_lst[:10]))
print('-'*50)
print(f"Sample questions above out of total No of questions:{len(question_lst)}")

--------------------------------------------------
What do you make of this Game of Thrones theory?
Should I learn AngularJS 1.5 instead of AngularJS 2 if I don't know TypeScript?
Are chatbots the new apps?
What is the fastest way to get a PAN card?
Who is the most inspiring writer of all time?
What are startups?
Could small talk be used to test someone's intelligence?
What is the strongest structure or strongest shape under compression?
Which are the best universities in the USA for doing a MS in Computer Science?
Which type of foundation is used in Indo-pak border?
--------------------------------------------------
Sample questions above out of total No of questions:19413


In [25]:
### checking if we have access to GPU by checking value of cuda

device = 'cuda' if torch.cuda.is_available() else "cpu"

if device != 'cuda':
    print("No cuda available")

No cuda available


In [26]:
### using transformer model all-MiniLM-L6-v2 that maps vector space to 384 dimensional space
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

In [27]:
query = "where does the queen of england live?"
xq = model.encode(query)
print(xq.shape)

(384,)


In [28]:
pinecone_obj = Pinecone(api_key=pinecone_api_key)
dl_ai_index_name = utils.create_dlai_index_name('dl-ai')
print(dl_ai_index_name)

dl-ai-118d1-7f10-4f3c-a4a6-41d7caaf84cc:fx


In [31]:
print(utils.get_dlapi_key())
print(len(utils.get_dlapi_key()))

607118d1-7f10-4f3c-a4a6-41d7caaf84cc:fx
39


In [35]:
print(utils.get_dlapi_key()[-36:-3])

118d1-7f10-4f3c-a4a6-41d7caaf84cc


In [37]:
dl_ai_index_name = dl_ai_index_name[:-3]

if dl_ai_index_name in pinecone_obj.list_indexes():
    pinecone_obj.delete_index(dl_ai_index_name)

print(dl_ai_index_name)

dl-ai-118d1-7f10-4f3c-a4a6-41d7caaf8


In [39]:
pinecone_obj.create_index(
    name=dl_ai_index_name,
    dimension=model.get_sentence_embedding_dimension(),
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-east-1')
)

index = pinecone_obj.Index(dl_ai_index_name)
print(index)

In [40]:
## create embeddings and upsert to pinecone
batch_size = 200
vector_limit = 1000

questions = question_lst[:vector_limit]

import json

for i in tqdm(range(0, len(questions), batch_size)):
    i_end = min(i+batch_size, vector_limit)
    ids = [str(x) for x in range(i, i_end)]
    metadata = [{'text':text} for text in questions[i:i_end]]
    xq = model.encode(questions[i:i_end])
    records = zip(ids, xq, metadata)
    index.upsert(vectors=records)
    

  0%|          | 0/5 [00:00<?, ?it/s]

In [41]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1000}},
 'total_vector_count': 1000}

In [42]:
## run your query
def run_query(query):
    embedding = model.encode(query).tolist()
    results = index.query(top_k=10, vector=embedding, include_metadata=True, include_values=False)
    for result in results["matches"]:
        print(result)

In [43]:
run_query("Why is baking soda added to cake batter?")

{'id': '389',
 'metadata': {'text': 'How do you start a bakery?'},
 'score': 0.354893982,
 'values': []}
{'id': '715',
 'metadata': {'text': 'Why do I have burps that taste like rotten eggs?'},
 'score': 0.325933635,
 'values': []}
{'id': '371',
 'metadata': {'text': 'Where can I buy best quality customized cupcakes in '
                      'Gold Coast?'},
 'score': 0.30146718,
 'values': []}
{'id': '460',
 'metadata': {'text': 'Where can I get good quality cupcakes and a lot of '
                      'different flavor in Gold Coast?'},
 'score': 0.300098896,
 'values': []}
{'id': '867',
 'metadata': {'text': 'What is a Bose–Einstein condensate? What is the use of '
                      'it?'},
 'score': 0.274294883,
 'values': []}
{'id': '194',
 'metadata': {'text': 'What are some of the best side dishes for crab cakes?'},
 'score': 0.268377125,
 'values': []}
{'id': '49',
 'metadata': {'text': 'What can be substituted for cornstarch in a recipe?'},
 'score': 0.259615362,
 'values